In [5]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# from net import Net
# from pytorch.ppg_feature_dataset import FeatureDataset
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


class FeatureDataset(Dataset):
    def __init__(self, file_name):
        
        # read csv file and load row data into variables
        file_out = pd.read_csv('test.csv')
        x = file_out.iloc[0:200, 0:6].values
        y = file_out.iloc[0:200, 5].values
        
        # Feature Scaling
        sc = StandardScaler()
        x_train = sc.fit_transform(x)
        y_train = y
        
        # Converting to torch sensors
        self.X_train = torch.tensor(x_train, dtype=torch.float32)
        self.y_train = torch.tensor(y_train)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx]
    

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
feature_set = FeatureDataset('test.csv')
train_loader = torch.utils.data.DataLoader(feature_set, batch_size=10, shuffle=True)

#defining ANN architecture

model = Net()
# model.to(cur.device)

#defining loss function and optimizer

loss_func = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

#train

epochs = 5

for e in range(epochs):
    running_loss = 0
    #batch training
    for features, labels in train_loader:
        #forward pass
        
        #output = model(features)
        output = model(features[None, None, ...])
        loss = loss_func(output, labels)
        
        optimizer.zero_grad() #clear previous gradients
        loss.backward() #backward pass
        optimizer.step()
        
        running_loss += loss.item()
        
else:
    print(loss.item())

RuntimeError: Calculated padded input size per channel: (4 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size